In [1]:
from torchvision.models.video import r3d_18, R3D_18_Weights
import torch
import cv2
import numpy as np
from torchvision.io.video import read_video
import os

In [2]:
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 60
import os

In [3]:
def frames_extraction(video_path):

    frames_list = []

    # Read the Video File
    video_reader = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading the frame from the video.
        success, frame = video_reader.read()

        if not success:
            break

        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

        # Normalize the resized frame
        normalized_frame = resized_frame / 255

        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)


    video_reader.release()

    return frames_list

In [4]:
DATASET_DIR = "data"
 
CLASSES_LIST = ["not Shop Lifters","shop lifter"]

In [5]:
def create_dataset(data_dir = DATASET_DIR):

    features = []
    labels = []
    video_files_paths = []

    # Iterating through all the classes.
    for class_index, class_name in enumerate(CLASSES_LIST):

        print(f'Extracting Data of Class: {class_name}')

        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))

        # Iterate through all the files present in the files list.
        for file_name in files_list:
            if file_name[-5]=="1":
                continue
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)

            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:

                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    features = np.asarray(features)
    labels = np.array(labels)

    return features, labels, video_files_paths

In [6]:
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: not Shop Lifters
Extracting Data of Class: shop lifter


In [7]:
one_hot_encoded = torch.nn.functional.one_hot(torch.tensor(labels), 2)

In [9]:
features.shape

(303, 60, 64, 64, 3)

In [10]:
features = np.transpose(features, (0,4, 1, 2, 3))

In [11]:
features.shape

(303, 3, 60, 64, 64)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, one_hot_encoded, test_size=0.1, random_state=42)

In [13]:
del features, one_hot_encoded

In [14]:
X_train.shape

(272, 3, 60, 64, 64)

In [16]:
X_train[0].shape

(3, 16, 64, 64)

In [15]:
checkpoint = torch.load('moem_ckpt.pth')
model = checkpoint['model_architecture']
model.load_state_dict(checkpoint['model_state_dict'])

VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [13]:
class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    
    def forward(self, x):
        return x

In [17]:
model.avgpool = Identity()

In [18]:
for param in model.parameters():
    param.requires_grad = False

In [30]:
model.fc = torch.nn.Linear(in_features=16384, out_features=2, bias=True)

In [16]:
model.fc.weight.requires_grad = True

In [17]:
model.fc.bias.requires_grad = True

In [18]:
criterion = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.0001)
#scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=20, gamma=0.5)

In [19]:
model.float().to("cuda")

VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [20]:
from tqdm import tqdm
EPOCHS = 200
loss = []
val_loss = []
batch_size = 2

In [51]:
model.to("cuda")

VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [22]:

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0  
    num_batches = int(X_train.shape[0] / batch_size)
    
    with tqdm(total=num_batches, desc=f'Epoch {epoch + 1}/{EPOCHS}', unit='batch') as pbar:
        for i in range(num_batches):
            opt.zero_grad()
            y_hat = model((torch.tensor(X_train[i * batch_size:(i + 1) * batch_size]).float().to("cuda")))
            _, target_indices = torch.max(torch.tensor(y_train[i * batch_size:(i + 1) * batch_size]), dim=1)
            loss_value = criterion(y_hat.to("cpu"), target_indices)
            loss.append(loss_value.item())
            loss_value.backward()
            opt.step()
            
            running_loss += loss_value.item()
            pbar.update(1)  
        
        avg_loss = running_loss / num_batches
        print(f"Train Loss: {avg_loss}")
    
    model.eval()
    with torch.no_grad():
        y_hat = model(torch.tensor(X_test[:32]).float().to("cuda"))
        _, target_indices = torch.max(torch.tensor(y_test), dim=1)
        v_l = criterion(y_hat.to("cpu"), target_indices[:32])
        val_loss.append(v_l)
        print(f"Val Loss: {v_l.item()}")
        

final_avg_loss = np.mean(loss)
print(f"Final average loss over all epochs: {final_avg_loss}")


Epoch 1/200:   0%|          | 0/136 [00:00<?, ?batch/s]/tmp/ipykernel_142035/2548819318.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _, target_indices = torch.max(torch.tensor(y_train[i * batch_size:(i + 1) * batch_size]), dim=1)
Epoch 1/200: 100%|██████████| 136/136 [00:13<00:00, 10.45batch/s]
/tmp/ipykernel_142035/2548819318.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _, target_indices = torch.max(torch.tensor(y_test), dim=1)


Train Loss: 0.5204660057925674
Val Loss: 0.41729676723480225


Epoch 2/200: 100%|██████████| 136/136 [00:13<00:00, 10.44batch/s]


Train Loss: 0.5156936054665815
Val Loss: 0.41594603657722473


Epoch 3/200: 100%|██████████| 136/136 [00:13<00:00, 10.43batch/s]


...

Train Loss: 0.36005216667784706
Val Loss: 0.3634624183177948


Epoch 199/200: 100%|██████████| 136/136 [00:13<00:00, 10.37batch/s]


Train Loss: 0.3598430142258568
Val Loss: 0.3635493516921997


Epoch 200/200: 100%|██████████| 136/136 [00:13<00:00, 10.37batch/s]


Train Loss: 0.3596349793883479
Val Loss: 0.3636372685432434
Final average loss over all epochs: 0.4028286269122869


In [28]:
model.to("cpu")
torch.save({
    'model_state_dict': model.state_dict(),
    'model_architecture': model,  
}, 'moem_ckpt.pth')

In [13]:
import torch
checkpoint = torch.load('moem_ckpt.pth')
model = checkpoint['model_architecture']
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [30]:
def predict_video(video_file_path, SEQUENCE_LENGTH):
    l = []
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frames_list = []

    predicted_class_name = ''

    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)

    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        success, frame = video_reader.read()

        if not success:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

        # Normalize the resized frame.
        normalized_frame = resized_frame / 255

        # Appending the pre-processed frame into the frames list
        frames_list.append(normalized_frame)

    predicted_labels_probabilities = model(torch.tensor(np.transpose(np.expand_dims(frames_list, axis = 0), (0,4, 1, 2, 3))).float().to("cpu"))

    predicted_label = torch.argmax(predicted_labels_probabilities)


    print(predicted_label)
    video_reader.release()


for pth in os.listdir("data/not Shop Lifters"):
    input_video_file_path = "data/not Shop Lifters/"+pth
    predict_video(input_video_file_path, SEQUENCE_LENGTH)
    



tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)


In [26]:
a = model(torch.tensor(X_test).float().to("cpu"))

In [28]:
torch.argmax(a, axis=1) == torch.argmax(y_test.to("cpu"), axis=1)

tensor([ True,  True,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False,  True,  True, False,  True, False,  True,  True,  True,
         True])

In [30]:
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score

y_test_np = y_test.to("cpu").detach().numpy()
a_np = a.to("cpu").detach().numpy()

accuracy = accuracy_score(np.argmax(y_test_np, axis=1), np.argmax(a_np, axis=1))
print(f"accuracy: {accuracy}")

print(classification_report(np.argmax(y_test_np, axis=1), np.argmax(a_np, axis=1)))



accuracy: 0.9802631578947368
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        96
           1       1.00      0.95      0.97        56

    accuracy                           0.98       152
   macro avg       0.98      0.97      0.98       152
weighted avg       0.98      0.98      0.98       152

